# Information about this notebook

This notebook is taken from the [DNA Diffusion project to load the DHSs](https://github.com/pinellolab/DNA-Diffusion/blob/main/notebooks/filter_master.ipynb "The DNA Diffusion Project").
Then I have updated some of the codes:
* I have broken down the codes into more cells such that the download of data are separate from reading information. This ensures that we do not download data from remote sites everytime we want to run the codes.

* I also updated the code to download the matrix 'dat_bin_FDR01_hg38.txt.gz' using gdown. the method specified in original notebook does not work.

* Next, when collecting names of cells from the matrix into a list with format celltype_encodeID I contatenated cols 'Biosample name','System' and 'DCC Library ID' as opposed to only using cols 'Biosample name' and 'DCC Library ID'. The reson being that the col 'System' contains important information like 'Cardiovascular' and includes more information related to cardiac systems.

In [2]:
import requests
import gzip
import numpy as np
import pandas as pd
from Bio import SeqIO

/Users/shakuntalabaichoo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/shakuntalabaichoo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# Download Reference genome
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
!gzip -d hg38.fa.gz

--2024-03-03 15:45:28--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.198.53
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.198.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: 'hg38.fa.gz'

hg38.fa.gz          100%[===================>] 938.09M  2.22MB/s    in 8m 44s  

2024-03-03 15:54:14 (1.79 MB/s) - 'hg38.fa.gz' saved [983659424/983659424]



# Loading genome

In [4]:
GENOME_PATH = "hg38.fa"

In [5]:
class DataSource:
    # Sourced from https://github.com/meuleman/SynthSeqs/blob/main/make_data/source.py

    def __init__(self, data, filepath):
        self.raw_data = data
        self.filepath = filepath

    @property
    def data(self):
        return self.raw_data


class ReferenceGenome(DataSource):
    """Object for quickly loading and querying the reference genome."""

    @classmethod
    def from_path(cls, path):
        genome_dict = {record.id: str(record.seq).upper() for record in SeqIO.parse(path, "fasta")}
        return cls(genome_dict, path)

    @classmethod
    def from_dict(cls, data_dict):
        return cls(data_dict, filepath=None)

    @property
    def genome(self):
        return self.data

    def sequence(self, chrom, start, end):
        chrom_sequence = self.genome[chrom]

        assert end < len(chrom_sequence), (
            f"Sequence position bound out of range for chromosome {chrom}. "
            f"{chrom} length {len(chrom_sequence)}, requested position {end}."
        )
        return chrom_sequence[start:end]


genome = ReferenceGenome.from_path(GENOME_PATH)

In [ ]:
# Download data
!wget https://www.meuleman.org/DHS_Index_and_Vocabulary_metadata.tsv

In [6]:
# Last row is empty
DHS_Index_and_Vocabulary_metadata = pd.read_table('./DHS_Index_and_Vocabulary_metadata.tsv').iloc[:-1]
with pd.option_context('display.max_rows', 5, 'display.max_columns', None):
    display(DHS_Index_and_Vocabulary_metadata)

,library order,Biosample name,Vocabulary representative,DCC Experiment ID,DCC Library ID,DCC Biosample ID,DCC File ID,Altius Aggregation ID,Altius Library ID,Altius Biosample ID,Replicate indicators,System,Subsystem,Organ,Biosample type,Biological state,Germ layer,Description,Growth stage,Age,Sex,Ethnicity,Donor ID,Unique cellular condition,Used in Figure 1b,Biosample protocol,Experiment protocol,Library kit method,Library cleanup,DNaseI units/mL,Amount Nucleic Acid (ng),Nuclei count,Protease inhibitor,Library sequencing date,Reads used,DCC SPOT score,Per-biosample peaks,DHSs in Index
0,1.0,GM06990,NaN,ENCSR000EMQ,ENCLB435ZZZ,ENCBS057ENC,ENCFF983CTQ,AG5636,LN1203,DS7748,DS7784,Hematopoietic,Lymphoid,Blood,Lines,Immortalized,Mesoderm,Lymphoblastoid,Adult,41Y,F,NaN,NaN,0,NaN,Biosample protocol,Experiment protocol,NaN,Sucrose,NaN,50,NaN,NaN,2009-02-23,142681590.0,0.6790,83639.0,82918.0
1,2.0,HepG2,NaN,ENCSR000ENP,ENCLB480ZZZ,ENCBS114ENC,ENCFF419JVG,AG5635,LN1207,DS7764,DS7768,Hepatic,NaN,Liver,Cancer,Cancer,Endoderm,hepatocellular carcinoma,Child,15Y,M,NaN,NaN,1,NaN,Biosample protocol,Experiment protocol,NaN,Sucrose,NaN,50,NaN,NaN,2009-02-23,138826342.0,0.5858,89748.0,89235.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,732.0,fPlacenta,NaN,ENCSR552RKI,ENCLB423VBC,ENCBS565KNL,ENCFF084UVH,AG8805,LN45072C,DS37386C,NaN,Fetal Life Support,NaN,Placenta,Primary,Primary,Endoderm,placenta,Fetal,102D,U,NaN,H26550,1,NaN,Biosample protocol,Experiment protocol,Thruplex DNA-Seq Q,NaN,NaN,1.325,1050000.0,A+Sucrose,NaN,203699532.0,0.3869,107611.0,106022.0
732,733.0,fPlacenta,Placental / trophoblast,ENCSR552XJI,ENCLB711ZZZ,ENCBS723HLT,ENCFF593AWN,AG7450,LN45076C,DS37716C,NaN,Fetal Life Support,NaN,Placenta,Primary,Primary,Endoderm,Placenta,Fetal,56D,U,NaN,H26598,1,NaN,Biosample protocol,Experiment protocol,Thruplex DNA-Seq Q,NaN,NaN,0.972,1380000.0,A+Sucrose,NaN,206456483.0,0.4356,115898.0,114344.0


In [ ]:
# Contains a 733 row (biosample) x 16 (component) peak presence/abscence matrix (not a binary matrix)
# Used later to map component number within metadata dataframe and find proportion for given component
# Downloading basis
basis_array = requests.get("https://zenodo.org/record/3838751/files/2018-06-08NC16_NNDSVD_Basis.npy.gz?download=1")

with open('2018-06-08NC16_NNDSVD_Basis.npy.gz', 'wb') as f:
    f.write(basis_array.content)

!gzip -d 2018-06-08NC16_NNDSVD_Basis.npy.gz

In [7]:
# Converting npy file to csv
basis_array = np.load('2018-06-08NC16_NNDSVD_Basis.npy')
np.savetxt("2018-06-08NC16_NNDSVD_Basis.csv", basis_array, delimiter=",")

# Creating nmf_loadings matrix from csv
nmf_loadings = pd.read_csv('2018-06-08NC16_NNDSVD_Basis.csv', header=None)
nmf_loadings.columns = ['C' + str(i) for i in range(1, 17)]


# Joining metadata with component presence matrix
DHS_Index_and_Vocabulary_metadata = pd.concat([DHS_Index_and_Vocabulary_metadata, nmf_loadings], axis=1)

In [8]:
DHS_Index_and_Vocabulary_metadata.head()

,library order,Biosample name,Vocabulary representative,DCC Experiment ID,DCC Library ID,DCC Biosample ID,DCC File ID,Altius Aggregation ID,Altius Library ID,Altius Biosample ID,...,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16
0,1.0,GM06990,NaN,ENCSR000EMQ,ENCLB435ZZZ,ENCBS057ENC,ENCFF983CTQ,AG5636,LN1203,DS7748,...,0.000000,0.0,0.000000,0.102685,0.0,0.0,0.026774,0.000000,0.0,0.000000
1,2.0,HepG2,NaN,ENCSR000ENP,ENCLB480ZZZ,ENCBS114ENC,ENCFF419JVG,AG5635,LN1207,DS7764,...,0.193557,0.0,0.074557,0.095928,0.0,0.0,3.190564,0.416094,0.0,0.000000
2,3.0,hTH1,NaN,ENCSR000EQC,ENCLB591ZZZ,ENCBS345AAA,ENCFF575KOF,AG5634,LN1222,DS7840,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
3,4.0,Hela,NaN,ENCSR000ENO,ENCLB479ZZZ,ENCBS890POO,ENCFF503PAE,AG4219,LN1264,DS8200,...,0.155545,0.0,0.000000,0.000000,0.0,0.0,0.407768,0.113676,0.0,2.420549
4,5.0,CACO2,NaN,ENCSR000EMI,ENCLB422ZZZ,ENCBS391ENC,ENCFF977BRD,AG4218,LN1269,DS8235,...,0.131876,0.0,0.000000,0.000000,0.0,0.0,0.936955,0.000000,0.0,0.000000


In [9]:
COMPONENT_COLUMNS = [
    'C1',
    'C2',
    'C3',
    'C4',
    'C5',
    'C6',
    'C7',
    'C8',
    'C9',
    'C10',
    'C11',
    'C12',
    'C13',
    'C14',
    'C15',
    'C16',
]

DHS_Index_and_Vocabulary_metadata['component'] = (
    DHS_Index_and_Vocabulary_metadata[COMPONENT_COLUMNS].idxmax(axis=1).apply(lambda x: int(x[1:]))
)

# Creating sequence metadata dataframe

CAUTION NEXT CELL CAN TAKE UPWARDS OF 10 MINUTES TO RUN

In [ ]:
# File loaded from drive available from below link
mixture_array = requests.get("https://zenodo.org/record/3838751/files/2018-06-08NC16_NNDSVD_Mixture.npy.gz?download=1")

# Downloading mixture array that contains 3.5M x 16 matrix of peak presence/absence decomposed into 16 components
with open('2018-06-08NC16_NNDSVD_Mixture.npy.gz', 'wb') as f:
    f.write(mixture_array.content)
!gzip -d 2018-06-08NC16_NNDSVD_Mixture.npy.gz

In [10]:
# Turning npy file into csv
mixture_array = np.load('2018-06-08NC16_NNDSVD_Mixture.npy').T
np.savetxt("2018-06-08NC16_NNDSVD_Mixture.csv", mixture_array, delimiter=",")

# Creating nmf_loadings matrix from csv and renaming columns
nmf_loadings = pd.read_csv('2018-06-08NC16_NNDSVD_Mixture.csv', header=None, names=COMPONENT_COLUMNS)

In [ ]:
# Download data
!wget https://www.meuleman.org/DHS_Index_and_Vocabulary_hg38_WM20190703.txt.gz
!gunzip -d DHS_Index_and_Vocabulary_hg38_WM20190703.txt.gz

In [11]:
# Loading in DHS_Index_and_Vocabulary_metadata that contains the following information:
# seqname, start, end, identifier, mean_signal, numsaples, summit, core_start, core_end, component

# Loading sequence metadata
sequence_metadata = pd.read_table('./DHS_Index_and_Vocabulary_hg38_WM20190703.txt', sep='\t')

# Dropping component column that contains associated tissue rather than component number (We will use the component number from DHS_Index_and_Vocabulary_metadata)
sequence_metadata = sequence_metadata.drop(columns=['component'], axis=1)

# Join metadata with component presence matrix
df = pd.concat([sequence_metadata, nmf_loadings], axis=1, sort=False)

/var/folders/1w/414v9fyj3jg9fp1hfb7m06jh0000gn/T/ipykernel_8721/625281858.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  sequence_metadata = pd.read_table('./DHS_Index_and_Vocabulary_hg38_WM20190703.txt', sep='\t')


In [12]:
# Functions used to create sequence column
def sequence_bounds(summit: int, start: int, end: int, length: int):
    """Calculate the sequence coordinates (bounds) for a given DHS.
    https://github.com/meuleman/SynthSeqs/blob/main/make_data/process.py
    """
    half = length // 2

    if (summit - start) < half:
        return start, start + length
    elif (end - summit) < half:
        return end - length, end

    return summit - half, summit + half


def add_sequence_column(df: pd.DataFrame, genome, length: int):
    """
    Query the reference genome for each DHS and add the raw sequences
    to the dataframe.
    Parameters
    ----------
    df : pd.DataFrame
        The dataframe of DHS annotations and NMF loadings.
    genome : ReferenceGenome(DataSource)
        A reference genome object to query for sequences.
    length : int
        Length of a DHS.

    https://github.com/meuleman/SynthSeqs/blob/main/make_data/process.py
    """
    seqs = []
    for rowi, row in df.iterrows():
        l, r = sequence_bounds(row['summit'], row['start'], row['end'], length)
        seq = genome.sequence(row['seqname'], l, r)

        seqs.append(seq)

    df['sequence'] = seqs
    return df


# Recreating some of the columns from our original dataset
df['component'] = df[COMPONENT_COLUMNS].idxmax(axis=1).apply(lambda x: int(x[1:]))
df['proportion'] = df[COMPONENT_COLUMNS].max(axis=1) / df[COMPONENT_COLUMNS].sum(axis=1)
df['total_signal'] = df['mean_signal'] * df['numsamples']
df['proportion'] = df[COMPONENT_COLUMNS].max(axis=1) / df[COMPONENT_COLUMNS].sum(axis=1)
df['dhs_id'] = df[['seqname', 'start', 'end', 'summit']].apply(lambda x: '_'.join(map(str, x)), axis=1)
df['DHS_width'] = df['end'] - df['start']

# Creating sequence column
df = add_sequence_column(df, genome, 200)

# Changing seqname column to chr
df = df.rename(columns={'seqname': 'chr'})

# Reordering and unselecting columns
df = df[
    [
        'dhs_id',
        'chr',
        'start',
        'end',
        'DHS_width',
        'summit',
        'numsamples',
        'total_signal',
        'component',
        'proportion',
        'sequence',
    ]
]
df

,dhs_id,chr,start,end,DHS_width,summit,numsamples,total_signal,component,proportion,sequence
0,chr1_16140_16200_16170,chr1,16140,16200,60,16170,1,0.129388,1,0.855153,CGGGCATCCTGTGTGCAGATACTCCCTGCTTCCTCTCTAGCCCCCA...
1,chr1_51868_52040_51970,chr1,51868,52040,172,51970,1,0.080034,7,0.973545,GGCGACCCAGCGAGACTCCGCCTCAAAAAAAAAAAAAGAAGATTGA...
2,chr1_57280_57354_57350,chr1,57280,57354,74,57350,4,1.093002,8,1.000000,CTCAGTCATTCCGAACAATTCACACACTAAGATTACCCATGCTAAA...
3,chr1_66370_66482_66430,chr1,66370,66482,112,66430,8,1.469725,3,0.332213,ATATATAAATTATATAATATAATATATATTATATAATATAATATAT...
4,chr1_79100_79231_79150,chr1,79100,79231,131,79150,2,0.226098,7,0.501840,CATTTCTCCAAGGAGGAAATACCAGAGTCAATTCACAACCACTGCA...
...,...,...,...,...,...,...,...,...,...,...,...
3591893,chrY_56882540_56882719_56882610,chrY,56882540,56882719,179,56882610,1,0.038079,5,0.803229,TTCTTTTTTAACACCACCACAACACTCAAGGAAATGAAAGTAGGTA...
3591894,chrY_56882864_56882980_56882930,chrY,56882864,56882980,116,56882930,1,0.115489,5,0.742349,TCCTAATGCTATCCCTTCCCCCTCCCCCTACCCCACATTGCTCTTC...
3591895,chrY_56883733_56883960_56883830,chrY,56883733,56883960,227,56883830,5,2.456885,7,0.559734,GGGTTGGTAATGAGGAGCCAAGGATGACTCATTTTCAGGTTGGAGT...
3591896,chrY_56884440_56884580_56884510,chrY,56884440,56884580,140,56884510,1,0.053759,5,0.803229,AGCAAGAGCTTACAGAACAGGCAGCACCAAGTGGGAGCTGGGGGGC...


In [13]:
# save df as feather file so that we do not need run this everytime we are testing our code
# It uses pyarrow; if it is not installed, we must install it, using the following
#!pip install pyarrow
df.to_feather('dhs_index_vocab_seq_dataset.ftr')

In [30]:
# Downloading binary peak matrix file from https://www.meuleman.org/research/dhsindex/

# !wget https://drive.google.com/uc?export=download&id=1Nel7wWOWhWn40Yv7eaQFwvpMcQHBNtJ2 # does not work
# Instead use gdown
# Install gdown if it does not exist, using the next line
# !pip install gdown


import gdown

url = 'https://drive.google.com/uc?export=download&id=16wbuNmHnwsek3USWM04nR535vPavNZka'
output = 'dat_bin_FDR01_hg38.txt.gz'
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?export=download&id=16wbuNmHnwsek3USWM04nR535vPavNZka
From (redirected): https://drive.google.com/uc?export=download&id=16wbuNmHnwsek3USWM04nR535vPavNZka&confirm=t&uuid=155233db-c762-4510-96fb-7d596fffbc37
To: /Users/shakuntalabaichoo/Projects/ShakunJobApps/UHN/Computational Biologist Peter Munk Lab/DHS_Analysis/dat_bin_FDR01_hg38.txt.gz
100%|████████████████████████████████████████████████████████████| 415M/415M [03:14<00:00, 2.14MB/s]


'dat_bin_FDR01_hg38.txt.gz'

In [ ]:
# Extracting binary peak matrix file from zip
!gzip -d dat_bin_FDR01_hg38.txt.gz

In [14]:
# Opening file
binary_matrix = pd.read_table('./dat_bin_FDR01_hg38.txt', header=None)

In [ ]:
# Collecting names of cells into a list with format celltype_encodeID
'''
celltype_encodeID = [
    row['Biosample name'] + "_" + row['DCC Library ID'] for _, row in DHS_Index_and_Vocabulary_metadata.iterrows()
]

# Renaming columns using celltype_encodeID list
binary_matrix.columns = celltype_encodeID
'''
# I wish to add the system info as well because it contains important information

In [15]:
# Collecting names of cells into a list with format celltype_encodeID
celltype_encodeID = [
    row['Biosample name'] + "_" + row['System'].replace(" ", "_") + "_" + row['DCC Library ID'] for _, row in DHS_Index_and_Vocabulary_metadata.iterrows()
]

# Renaming columns using celltype_encodeID list
binary_matrix.columns = celltype_encodeID

In [16]:
master_dataset = pd.concat([df, binary_matrix], axis=1, sort=False)

In [17]:
master_dataset

,dhs_id,chr,start,end,DHS_width,summit,numsamples,total_signal,component,proportion,...,fKidney_Renal_ENCLB005SRL,fKidney_Renal_ENCLB704GMQ,fKidney_Renal_ENCLB759USM,fLung_Respiratory_ENCLB594BSZ,fKidney_Renal_ENCLB049MNH,fUmbilical_cord_Fetal_Life_Support_ENCLB771UER,fBone_femur_Musculoskeletal_ENCLB236BWV,fLiver_Hepatic_ENCLB638FEH,fPlacenta_Fetal_Life_Support_ENCLB423VBC,fPlacenta_Fetal_Life_Support_ENCLB711ZZZ
0,chr1_16140_16200_16170,chr1,16140,16200,60,16170,1,0.129388,1,0.855153,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1,chr1_51868_52040_51970,chr1,51868,52040,172,51970,1,0.080034,7,0.973545,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
2,chr1_57280_57354_57350,chr1,57280,57354,74,57350,4,1.093002,8,1.000000,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
3,chr1_66370_66482_66430,chr1,66370,66482,112,66430,8,1.469725,3,0.332213,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
4,chr1_79100_79231_79150,chr1,79100,79231,131,79150,2,0.226098,7,0.501840,...,0.0,0.0,0.0,0.12227,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3591893,chrY_56882540_56882719_56882610,chrY,56882540,56882719,179,56882610,1,0.038079,5,0.803229,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
3591894,chrY_56882864_56882980_56882930,chrY,56882864,56882980,116,56882930,1,0.115489,5,0.742349,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
3591895,chrY_56883733_56883960_56883830,chrY,56883733,56883960,227,56883830,5,2.456885,7,0.559734,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
3591896,chrY_56884440_56884580_56884510,chrY,56884440,56884580,140,56884510,1,0.053759,5,0.803229,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Save as feather file - it uses pyarrow; if it does not exist, we must install it, using the following
#!pip install pyarrow

master_dataset.to_feather('master_dataset.ftr')